# Setup

In [1]:
import os
import sys
sys.path.append("..")

debug = True # If True, will produce plots and/or verbose output to double-check

print(f"Current working directory: {sys.path[0]}")
%run -i "../parameters/parameters.py"
%run -i "../src/path.py"
%run -i "../src/setup.py"
%run -i "../src/functions.py"
%run -i "greedy_triangulation.py"
%run -i "graph_similarity.py"
%run -i "write_results_percentages.py"
%run -i "analyze_poi_based_results.py"

%load_ext watermark
%watermark -n -v -m -g -iv

Current working directory: /Users/jaborup/Projects/continuous-bike-growth
Loaded parameters.

Loaded PATH.



=== Cities ===
{   'bern': {'countryid': 'che', 'name': 'Bern', 'nominatimstring': 'bern'},
    'copenhagen': {   'countryid': 'dnk',
                      'name': 'Copenhagen',
                      'nominatimstring': ''},
    'montreal': {   'countryid': 'can',
                    'name': 'Montreal',
                    'nominatimstring': 'montreal (06)'}}


Setup finished.

Loaded functions.

osgeo.gdal 3.8.5
matplotlib 3.8.4
shapely    2.0.4
networkx   3.3
igraph     0.11.6
fiona      1.9.5
csv        1.0
watermark  2.0.2
numpy      1.26.4
pyproj     3.6.1
geopandas  1.0.1
osmnx      2.0.0b3
pandas     2.2.3
geojson    2.5.0
Thu Nov 28 2024 

CPython 3.12.7
IPython 8.6.0

compiler   : Clang 17.0.6 
system     : Darwin
release    : 23.6.0
machine    : arm64
processor  : arm
CPU cores  : 8
interpreter: 64bit
Git hash   : ef7f852c24da293a7da2afa6db6d86ab48e5b0df


In [2]:
city = "copenhagen"
iterations = 15

prune_quantiles = [0.5]
subgraph_percentages_tests = [
    [0.6, 0.4],
]

# Run the program

In [3]:
placeid = city
print(f"{placeid}: Generating graph and POIs")

G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
    nnids = [int(line.rstrip()) for line in f]

for prune_measure in ["betweenness", "random", "closeness"]:
    print(f"Prune measure: {prune_measure}")

    (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

    for subgraph_percentages in subgraph_percentages_tests:
        file_name = f"{placeid}_{prune_measure}_{'_'.join(map(str, subgraph_percentages))}.csv"
        print(f"Writing data to {file_name}")
        dir = "csv_data/"
        path = os.path.join(dir, file_name)
        with open(path, 'w') as f:
            writer = csv.writer(f)
            writer.writerow([
                "sum_of_errors",
                "continuous_vertices",
                "bikengrowth_vertices",
                "continuous_disconnected_pairs",
                "bikengrowth_disconnected_pairs",
            ])

            for i in tqdm(range(iterations), desc="Iterations", leave=False):
                if prune_measure == "random":
                    (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

                gts, abstract_gts = greedy_triangulation_in_steps(G_carall, nnids, subgraph_percentages, prune_quantiles, prune_measure)

                for prune_quantile, gt, szell_gt in zip(prune_quantiles, gts, GTs):
                    error, szell_disconnected_points, our_disconnected_points = sum_of_errors_pois(szell_gt, gt, nnids)

                writer.writerow([error, gt.vcount(), szell_gt.vcount(), our_disconnected_points, szell_disconnected_points])
                # write_result_with_percentages(res=results, mode="pickle", placeid=placeid, iteration=str(i), poi_source=poi_source, prune_measure=prune_measure, suffix=".pickle")

copenhagen: Generating graph and POIs


/Users/jaborup/Projects/continuous-bike-growth/src/functions.py:346: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  e = pd.read_csv(p + prefix + '_edges.csv')


Prune measure: closeness


Greedy triangulation:   0%|          | 0/1 [00:00<?, ?it/s]

Writing data to copenhagen_closeness_0.25_0.25_0.25_0.25.csv


...

Stepwise greedy triangulation on subgraphs:   0%|          | 0/1 [00:00<?, ?it/s]

Stepwise greedy triangulation on subgraphs:   0%|          | 0/1 [00:00<?, ?it/s]

Stepwise greedy triangulation on subgraphs:   0%|          | 0/1 [00:00<?, ?it/s]

Stepwise greedy triangulation on subgraphs:   0%|          | 0/1 [00:00<?, ?it/s]

Stepwise greedy triangulation on subgraphs:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Analyze results
analyze_poi_base_results_iterations(placeid=placeid, iterations=iterations)